In [132]:
import numpy as np
import pandas as pd
import os
import os.path as opath
# import ast
# import json
#from pandas.io.json import json_normalize
import datetime
import pickle
import math

# import matplotlib as plt
# import matplotlib.pyplot as plt
# import seaborn as sb
import plotly 
import plotly.plotly as py
import plotly.presentation_objs as pres
from plotly.graph_objs import *
#import plotly.graph_objs as go
import plotly.offline as offline
from plotly import tools
from IPython.display import Image

from IPython.core.display import HTML
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
plotly.tools.set_credentials_file(username='saurabhpundir', api_key='evTecysAK8CEVYBe6JUW')

<div class="span5 alert alert-info">
<h2>Note book details</h2>

<p> This notebook is for <b>data story</b> for <b>Mortgage client segmentation</b> project.</p>

<p> Notes.</p>
<ol>
<li>  The file will utlize the pickle file created in third part of for data wrangling</li> 
<li>  The file will perform exploratory analysis on the mortgage data with some </li> 
        
</ol>
</div>

In [133]:
# from IPython.core.display import HTML
css = open('style/style-table.css').read() + open('style/style-notebook.css').read()
cssAnalysis = '.analysis {background:#5bffc0; }'
HTML('<style>{} {}</style>'.format(css,cssAnalysis))
pickle_file='pickle_KMean_All_Col_CA_Cluster_3.sa'

colorApprove='rgb(50,205,50)'
colorDeny='rgb(255,0,0)'
colorOrange1='rgb(254, 80, 6)'#'rgb(255, 112, 67)'#'rgb(255, 87, 34)'# 1
colorBlue2='rgb(26, 4, 226)' # 2
colorGreen3='rgb(76, 175, 80)'#3
colorYellow4='rgb(255, 235, 59)' #4
colorRed5='rgb(255,0,0)' #5
colorGray='rgb(190,190,190)' # unknown
colorPink='rgb(255,105,180)'
colorCluster1=colorOrange1# colorYellow4
colorCluster2=colorBlue2
colorCluster3=colorGreen3

In [134]:
#'CountyCode'
# 'ApplicantSex'
# 'ApplicantEthnicity'
# 'ApplicantRace'
# 'Occupancy'
# 'PropertyType'
# 'LoanPurpose'
# 'LoanAmount'
# 'ApplicantIncome'
# County graph for cluster or loan amount

In [135]:
df_selectdata = pickle.load( open( pickle_file, "rb" ) )
#df_alldata = pickle.load( open( 'df_alldata_hmda_cenus.sa', "rb" ) )
#df_selectdata.info()

In [136]:
# remove some without county data(31 rows)
df_selectdata=df_selectdata[~df_selectdata.CountyName.isna()]

In [137]:
df_selectdata.cluster_label=df_selectdata.cluster_label.apply(lambda X:X+1)

In [138]:
#"Santa Clara bounty". ("County")

In [139]:
def removeCountytext(strInput,lookup="County"):
    if pd.isna(strInput):
        return
    #print(strInput)
    if lookup in strInput: 
        return strInput[:strInput.index(lookup)]
    else:
        return strInput
    
df_selectdata.CountyName=df_selectdata.CountyName.apply(lambda s: removeCountytext(s))

In [140]:
#df_selectdata[df_selectdata.CountyName.isnull()]
#tCounty=df_selectdata['CountyName'].apply(lambda s: s[:s.index("County")])
Countylist=df_selectdata['CountyName'].unique()
#Countylist
Clusterlist=df_selectdata.cluster_label.unique()

In [141]:
#Countylist
Clusterlist

array([1, 2, 3], dtype=int64)

In [142]:
df_selectdata.groupby('cluster_label').index.count()

cluster_label
1    52180
2    35683
3    12099
Name: index, dtype: int64

In [143]:
#df_selectdata[df_selectdata.CountyName.isna()]
#df_selectdata.groupby(['CountyName','cluster_label']).count()

In [144]:
### helper methods

In [145]:
def getAllcolNameforDataframe(df, isSep=False):
    rownum=0
    for col in df:
        if(isSep):
            print("'"+str(col)+"',")
        else:
            print(str(rownum)+":"+str(col))
        rownum=rownum+1

In [146]:
def getCountForAmountRange(df,colname,min,max,grpbycol='ActionType',outputcol='Year'):
    return pd.DataFrame(df[(df[colname]>min) & (df[colname]<=max)].groupby(grpbycol)[outputcol].count())
    

In [147]:
def checkforNullCol(colname,df):
    return df[df[colname].isnull()]

In [148]:
def getDataFrameGrroupedbyCol(dfselect,colname):
    dfgroup=dfselect.groupby([colname,'Result'], as_index=False)
    dfgraph=dfgroup.count().loc[:,[colname,'Result','Year']]
    dfgraph=dfgraph[dfgraph['Year'].isna()==False]
    dfgraph['Count']=((dfgraph['Year']/1000.0)).apply(math.ceil)
    #dfgraph_col=dfgraph_col.assign(Count=((dfgraph['Year']/1000.0)).apply(math.ceil))
    return dfgraph
    

In [149]:
def getDataFrameGrroupedbyClusterAndCol(dfselect,colname):
    dfgroup=dfselect.groupby([colname,'cluster_label'], as_index=False)
    dfgraph=dfgroup.count().loc[:,[colname,'cluster_label','Year']]
    dfgraph=dfgraph[dfgraph['Year'].isna()==False]
    dfgraph['Count']=((dfgraph['Year']/1000.0)).apply(math.ceil)
    #dfgraph_col=dfgraph_col.assign(Count=((dfgraph['Year']/1000.0)).apply(math.ceil))
    return dfgraph

In [150]:
def getlistbycolumnForCluster(dfselect,colname,value,fulllist=False):
    if ( type(value) == int):
        dfgraph_col=dfselect[dfselect[colname]==value]
    else:
        dfgraph_col=dfselect[dfselect[colname].isin(value)]
    #dfgraph_female=dfgraph[dfgraph.ApplicantSex==2]
    dfgraph_col=dfgraph_col.assign(perc=((dfgraph_col['Count']/dfgraph_col['Count'].sum())*100))
    templist=dfgraph_col.groupby('Result').sum().sort_values('Result').perc.tolist()

    percListtext=list(map((lambda x:str(round(x))+'%'),templist))
    if(fulllist):
        percList=list(map((lambda x:round(x)),templist))
    else:
        percList=list(map((lambda x:round(x/2)),templist))
        percList.insert(0, 50)
        percListtext.insert(0, 0)

    return percList,percListtext


In [151]:
def getlistbycolumnForCluster(dfselect,colname,value,fulllist=False):
    if ( type(value) == int):
        dfgraph_col=dfselect[dfselect[colname]==value]
    else:
        dfgraph_col=dfselect[dfselect[colname].isin(value)]
    #dfgraph_female=dfgraph[dfgraph.ApplicantSex==2]
    dfgraph_col=dfgraph_col.assign(perc=((dfgraph_col['Count']/dfgraph_col['Count'].sum())*100))
    templist=dfgraph_col.groupby('cluster_label').sum().sort_values('cluster_label').perc.tolist()

    percListtext=list(map((lambda x:str(round(x))+'%'),templist))
    if(fulllist):
        percList=list(map((lambda x:round(x)),templist))
    else:
        percList=list(map((lambda x:round(x/2)),templist))
        percList.insert(0, 50)
        percListtext.insert(0, 0)

    return percList,percListtext


In [152]:
def getlistofResultbycolumn(dfselect,value,fulllist=True):
    if (type(value) == int):
        dfgraph_col=dfselect[dfselect.Result==value]
    else:
        dfgraph_col=dfselect[dfselect.Result.isin(value)]
    dfgraph_col=dfgraph_col.assign(perc=((dfgraph_col['Count']/dfgraph_col['Count'].sum())*100))
    #templist=dfgraph_col.groupby('Result').sum().sort_values('Result').perc.tolist()
    templist=dfgraph_col.sort_values('Result').perc.tolist()

    percListtext=list(map((lambda x:str(round(x))+'%'),templist))
    if(fulllist):
        percList=list(map((lambda x:round(x)),templist))
    else:
        percList=list(map((lambda x:round(x/2)),templist))
        percList.insert(0, 50)
        percListtext.insert(0, 0)

    return percList,percListtext


In [153]:
def getlistofCusterbycolumn(dfselect,value,fulllist=True,percData=False):
    if (type(value) == np.int64):
        dfgraph_col=dfselect[dfselect.cluster_label==value]
    else:
        dfgraph_col=dfselect[dfselect.cluster_label.isin(value)]
    if(percData):
                
        dfgraph_col=dfgraph_col.assign(perc=((dfgraph_col['Count']/dfgraph_col['Count'].sum())*100))
        #templist=dfgraph_col.groupby('Result').sum().sort_values('Result').perc.tolist()
        templist=dfgraph_col.sort_values('cluster_label').perc.tolist()

        percListtext=list(map((lambda x:str(round(x))+'%'),templist))
        if(fulllist):
            percList=list(map((lambda x:round(x)),templist))
        else:
            percList=list(map((lambda x:round(x/2)),templist))
            percList.insert(0, 50)
            percListtext.insert(0, 0)
        return percList,percListtext
    else:
        #templist=dfgraph_col.sort_values('cluster_label').Count.tolist()
        templist=dfgraph_col.sort_values('cluster_label').Year.tolist()
        numListtext=list(map((lambda x:str(round(x))+"(1000's)"),templist))
        if(fulllist):
            numList=list(map((lambda x:round(x)),templist))
        else:
            numList=list(map((lambda x:round(x/2)),templist))
            numList.insert(0, 50)
            numList.insert(0, 0)
        return numList,numListtext

In [173]:
def createofflineChart(fig,filenameval,image=False,displayonpage=False):
#def createofflineChart(fig,filenameval,image=True,displayonpage=True):
    if(image):
        #offline.Image.plot(fig, filename=filenameval,image='png',image_filename=filenameval,output_type='file')
        offline.plot(fig, auto_open=True, image = 'png', image_filename=filenameval,
             output_type='file', #image_width=800, image_height=600, 
             filename=filenameval+'.html', validate=False)
        #py.image.save_as(fig, filename=filenameval+'.png')
        #Image(filenameval+'.png')
    else:
        offline.plot(fig, filename=filenameval+'.html')
#     if(displayonpage):
#         Image(filenameval+'.png')

In [155]:
def createbasemeterForDonutWith3Section(domainval,inputPercList,inputPercListText,labellist):
    base_chart = {
        "values": [40, 10, 10, 10, 10, 10, 10],
        "labels": ["-", "0", "20", "40", "60", "80", "100"],
       'domain':domainval,
        "marker": {
            "colors": [
                'rgb(255, 255, 255)',
                'rgb(255, 255, 255)',
                'rgb(255, 255, 255)',
                'rgb(255, 255, 255)',
                'rgb(255, 255, 255)',
                'rgb(255, 255, 255)',
                'rgb(255, 255, 255)'
            ],
            "line": {
                "width": 1
            }
        },
        "name": "Gauge",
        "hole": .4,
        "type": "pie",
        "direction": "clockwise",
        "rotation": 108,
        "showlegend": False,
        "hoverinfo": "none",
        "textinfo": "text",
        "textposition": "outside"
    }
    meter_chart = {
        "values": inputPercList,
        "labels":labellist,
        "text":inputPercListText,
        "marker": {
            'colors': [
                'rgb(255, 255, 255)',
                colorCluster1,
                 colorCluster2,
                colorCluster3
            ]
        },
        "domain": domainval,
        "name": "Gauge",
        "hole": .3,
        "type": "pie",
        "direction": "clockwise",
        "rotation": 90,
        "showlegend": True,
        "textinfo": "text",
        "textposition": "inside",
         "hoverinfo": "none",
    }
    # we don't want the boundary now
    base_chart['marker']['line']['width'] = 0
#     print(inputPercList)
#     print(labellist)
#     print(inputPercListText)
#     print(domainval)
#     print(meter_chart)
    return base_chart,meter_chart


In [156]:
def createHalfDonut(title,inputPercList,inputPercListText,labellist):
    base_chart,meter_chart=createbasemeterForDonutWith3Section({"x": [0, .48]},inputPercList,inputPercListText,labellist)
    layout = {
        'title':title,
        'xaxis': {
            'showticklabels': False,
            'showgrid': False,
            'zeroline': False,
        },
        'yaxis': {
            'showticklabels': False,
            'showgrid': False,
            'zeroline': False,
        },
        'shapes': [
            {
                'type': 'path',
                'path': 'M 0.235 0.5 L 0.24 0.65 L 0.245 0.5 Z',
                'fillcolor': 'rgba(44, 160, 101, 0.5)',
                'line': {
                    'width': 0.5
                },
                'xref': 'paper',
                'yref': 'paper'
            }
        ],
        'annotations': [
            {
                'xref': 'paper',
                'yref': 'paper',
                'x': 0.23,
                'y': 0.45,
                'text': '50',
                'showarrow': False
            }
        ]
    }
    return base_chart, meter_chart,layout 

In [157]:
def createThreeHalfDonut(title,inputPercList1,inputPercListText1,inputPercList2,inputPercListText2,
                        inputPercList3,inputPercListText3,labellist,filenameval):
   
    inputPercListText1[0]=labellist[0]
    inputPercListText2[0]=labellist[1]
    inputPercListText3[0]=labellist[2]

    
    #labels=["","Denied","Accepted"]
    labels=[]
    labels.append('')
    labels.extend(Clusterlist.astype('str'))
    #print(labels)
    domain1={'x':[0, 0.33], 'y':[0.66,1.0]}# for the cell (1,1)
    domain2={'x':[0.66, 1], 'y':[0.66,1]}#cell (1,2)
    domain3={'x':[0, 0.33], 'y':[0.33, 0.66]}#cell (2,1)

    base_chart1,meter_chart1=createbasemeterForDonutWith3Section(domain1,inputPercList1,inputPercListText1,labels)
    base_chart2,meter_chart2=createbasemeterForDonutWith3Section(domain2,inputPercList2,inputPercListText2,labels)
    base_chart3,meter_chart3=createbasemeterForDonutWith3Section(domain3,inputPercList3,inputPercListText3,labels)
    
    layout = {
        'autosize':False,
        'height' : 1000,
        'width' : 900,
        'title':title,
        'xaxis': {
            'showticklabels': False,
            'showgrid': False,
            'zeroline': False,
        },
        'yaxis': {
            'showticklabels': False,
            'showgrid': False,
            'zeroline': False,
        }#,
#         'shapes': [
#             {
#                 'type': 'path',
#                 'path': 'M 0.235 0.5 L 0.24 0.65 L 0.245 0.5 Z',
#                 'fillcolor': 'rgba(44, 160, 101, 0.5)',
#                 'line': {
#                     'width': 0.5
#                 },
#                 'xref': 'paper',
#                 'yref': 'paper'
#             }
#         ],
#         'annotations': [
#             {
#                 'xref': 'paper',
#                 'yref': 'paper',
#                 'x': 0.23,
#                 'y': 0.45,
#                 'text': '50',
#                 'showarrow': False
#             }
#        ]
    }
#     fig = {"data": [base_chart1, base_chart2,base_chart3, base_chart4,
#                     meter_chart1,meter_chart2, meter_chart3, meter_chart4],
    fig = {"data": [
                    meter_chart1,meter_chart2, meter_chart3],
           "layout": layout }
    createofflineChart(fig,filenameval)

In [158]:
def createFourHalfDonut(title,inputPercList1,inputPercListText1,inputPercList2,inputPercListText2,
                        inputPercList3,inputPercListText3,inputPercList4,inputPercListText4,labellist,filenameval):
    
    #labels=["","Denied","Accepted"]
    labels=[]
    labels.append('')
    labels.extend(Clusterlist.astype('str'))
    #print(labels)
    inputPercListText1[0]=labellist[0]
    inputPercListText2[0]=labellist[1]
    inputPercListText3[0]=labellist[2]
    inputPercListText4[0]=labellist[3]
    
    domain1={'x': [0, .48],'y': [0, .49]}# for the cell (1,1)
    domain2={'x': [.51, 1],'y': [0, .49]}#cell (1,2)
    domain3={'x': [0, .48],'y': [.51, 1]}#cell (2,1)
    domain4={'x': [.51, 1],'y': [.51, 1]}#cell (2,2)

    base_chart1,meter_chart1=createbasemeterForDonutWith3Section(domain1,inputPercList1,inputPercListText1,labels)
    base_chart2,meter_chart2=createbasemeterForDonutWith3Section(domain2,inputPercList2,inputPercListText2,labels)
    base_chart3,meter_chart3=createbasemeterForDonutWith3Section(domain3,inputPercList3,inputPercListText3,labels)
    base_chart4,meter_chart4=createbasemeterForDonutWith3Section(domain4,inputPercList4,inputPercListText4,labels)

    layout = {
        'title':title,
        'xaxis': {
            'showticklabels': False,
            'showgrid': False,
            'zeroline': False,
        },
        'yaxis': {
            'showticklabels': False,
            'showgrid': False,
            'zeroline': False,
        }#,
#         'shapes': [
#             {
#                 'type': 'path',
#                 'path': 'M 0.235 0.5 L 0.24 0.65 L 0.245 0.5 Z',
#                 'fillcolor': 'rgba(44, 160, 101, 0.5)',
#                 'line': {
#                     'width': 0.5
#                 },
#                 'xref': 'paper',
#                 'yref': 'paper'
#             }
#         ],
#         'annotations': [
#             {
#                 'xref': 'paper',
#                 'yref': 'paper',
#                 'x': 0.23,
#                 'y': 0.45,
#                 'text': '50',
#                 'showarrow': False
#             }
#        ]
    }
    fig = {"data": [
                    meter_chart1,meter_chart2, meter_chart3, meter_chart4],
           "layout": layout }
    createofflineChart(fig,filenameval)

In [159]:
def createSixHalfDonut(title,inputPercList1,inputPercListText1,inputPercList2,inputPercListText2,
                        inputPercList3,inputPercListText3,inputPercList4,inputPercListText4,inputPercList5,
                       inputPercListText5,inputPercList6,inputPercListText6,labellist,filenameval):
   
    inputPercListText1[0]=labellist[0]
    inputPercListText2[0]=labellist[1]
    inputPercListText3[0]=labellist[2]
    inputPercListText4[0]=labellist[3]
    inputPercListText5[0]=labellist[4]
    inputPercListText6[0]=labellist[5]
    
    #labels=["","Denied","Accepted"]
    labels=[]
    labels.append('')
    labels.extend(Clusterlist.astype('str'))
    #print(labels)
    
    domain1={'x':[0, 0.33], 'y':[0.66,1.0]}# for the cell (1,1)
    domain2={'x':[0.66, 1], 'y':[0.66,1]}#cell (1,2)
    domain3={'x':[0, 0.33], 'y':[0.33, 0.66]}#cell (2,1)
    domain4={'x':[0.66, 1], 'y':[0.33, 0.66]}#cell (2,2)
    domain5={'x':[0, 0.33], 'y':[0,0.33]}#cell (3,1)
    domain6={'x': [0.66, 1], 'y':[0,0.33]}#cell (3,1)

#     domain1={'x': [0, .48],'y': [0, .32]}# for the cell (1,1)
#     domain2={'x': [.52, 1],'y': [0, .32]}#cell (1,2)
#     domain3={'x': [0, .48],'y': [.34, .65]}#cell (2,1)
#     domain4={'x': [.52, 1],'y': [.34, .65]}#cell (2,2)
#     domain5={'x': [0, .48],'y': [.67, 1]}#cell (3,1)
#     domain6={'x': [.52, 1],'y': [.67, 1]}#cell (3,1)

    #domain1={'x': [0, .48],'y': [0, .32]}
    base_chart1,meter_chart1=createbasemeterForDonutWith3Section(domain1,inputPercList1,inputPercListText1,labels)
    #print(inputPercListText1[0])
    #domain2={'x': [.52, 1],'y': [0, .32]}
    base_chart2,meter_chart2=createbasemeterForDonutWith3Section(domain2,inputPercList2,inputPercListText2,labels)
    #print(inputPercListText2[0])
    #domain3={'x': [0, .48],'y': [.34, .65]}
    base_chart3,meter_chart3=createbasemeterForDonutWith3Section(domain3,inputPercList3,inputPercListText3,labels)
    #print(inputPercListText3[0])
    #domain4={'x': [.52, 1],'y': [.34, .65]}
    base_chart4,meter_chart4=createbasemeterForDonutWith3Section(domain4,inputPercList4,inputPercListText4,labels)
    #print(inputPercListText4[0])
    #domain5={'x': [0, .48],'y': [.67, 1]}
    base_chart5,meter_chart5=createbasemeterForDonutWith3Section(domain5,inputPercList5,inputPercListText5,labels)
    #print(inputPercListText5[0])
    #domain6={'x': [.52, 1],'y': [.67, 1]}
    base_chart6,meter_chart6=createbasemeterForDonutWith3Section(domain6,inputPercList6,inputPercListText6,labels)
#     print(inputPercListText6[0])

#     fig = tools.make_subplots(rows=3, cols=2, subplot_titles=('Plot 1', 'Plot 2',
#                                                           'Plot 3', 'Plot 4', 'Plot 5', 'Plot 6'))
    
#     fig.append_trace(meter_chart1, 1, 1)
#     fig.append_trace(meter_chart2, 1, 2)
#     fig.append_trace(meter_chart3, 2, 1)
#     fig.append_trace(meter_chart4, 2, 2)
#     fig.append_trace(meter_chart3, 3, 1)
#     fig.append_trace(meter_chart4, 3, 2)
    
    layout = {
        'autosize':False,
        'height' : 1000,
        'width' : 900,
        'title':title,
        'xaxis': {
            'showticklabels': False,
            'showgrid': False,
            'zeroline': False,
        },
        'yaxis': {
            'showticklabels': False,
            'showgrid': False,
            'zeroline': False,
        }#,
#         'shapes': [
#             {
#                 'type': 'path',
#                 'path': 'M 0.235 0.5 L 0.24 0.65 L 0.245 0.5 Z',
#                 'fillcolor': 'rgba(44, 160, 101, 0.5)',
#                 'line': {
#                     'width': 0.5
#                 },
#                 'xref': 'paper',
#                 'yref': 'paper'
#             }
#         ],
#         'annotations': [
#             {
#                 'xref': 'paper',
#                 'yref': 'paper',
#                 'x': 0.23,
#                 'y': 0.45,
#                 'text': '50',
#                 'showarrow': False
#             }
#        ]
    }
#     fig = {"data": [base_chart1, base_chart2,base_chart3, base_chart4,
#                     meter_chart1,meter_chart2, meter_chart3, meter_chart4],
    fig = {"data": [
                    meter_chart1,meter_chart2, meter_chart3, meter_chart4, meter_chart5, meter_chart6],
           "layout": layout }
    #fig = {"data": [meter_chart1, meter_chart2, meter_chart3, meter_chart4]}
    #py.iplot(fig, filename='CA-Cluster-half-chart-Male-County')
    #offline.plot(fig, filename=filenameval)
    createofflineChart(fig,filenameval)
   
    #return base_chart, meter_chart,layout 

In [160]:
# graph for 0-1101
# graph for 1101-5101
# graph for 5101-99999

def drawIncomegraphonRange(df,title,start, end,selcolor):
    
    #start=0 
    #end=1101    
    dfgraph=df[(np.int64(df.start)>=np.int64(start))& (np.int64(df.end)<=np.int64(end))].sort_values(by=['start'])
    #dfgraph=df_income_range[(np.int64(df_income_range.start)>np.int64(start)]
    #dfgraph=df_income_range[(np.int64(df_income_range.start)>np.int64(start))]                         
    #dfgraph=df_income_range[(np.int64(df_income_range.end)<=np.int64(end))].sort_values(by=['start'])
    
    traceApprove =graph_objs.Bar( 
#       x= dfgraph[dfgraph['ActionType']==1].range, 
#       y= dfgraph[dfgraph['ActionType']==1].value, 
      x= dfgraph.range, 
      y= dfgraph.value, 
        
      name= "Loan Amount",
          #orientation = 'h',
    marker=dict(
            color=selcolor,
            line=dict(
                color=selcolor,
                width=1.5,
            )
        ),
        opacity=0.6,

    )
    data = Data([traceApprove])
#     traceNotAccepted = graph_objs.Bar(
#       x= dfgraph[dfgraph['ActionType']==2].range, 
#       y= dfgraph[dfgraph['ActionType']==2].value,
#       name= "Application approved but not accepted", 
#           #orientation = 'h',
#     marker=dict(
#             color='rgb(255,255,0)',
#             line=dict(
#                 color='rgb(255,255,0)',
#                 width=1.5,
#             )
#         ),
#         opacity=0.6
#     )
#     traceDenied = graph_objs.Bar(
#       x= dfgraph[dfgraph['ActionType']==3].range, 
#       y= dfgraph[dfgraph['ActionType']==3].value,
#       name= "Application denied by financial institution", 
#           #orientation = 'h',
#     marker=dict(
#             color='rgb(255,0,16)',
#             line=dict(
#                 color='rgb(255,0,16)',
#                 width=1.5,
#             )
#         ),
#         opacity=0.6
#     )
#     tracePurchased = graph_objs.Bar(
#       x= dfgraph[dfgraph['ActionType']==6].range, 
#       y= dfgraph[dfgraph['ActionType']==6].value,
#       name= "Loan purchased by your institution", 
#           #orientation = 'h',
#     marker=dict(
#             color='rgb(190,190,190)',
#             line=dict(
#                 color='rgb(119,136,153)',
#                 width=1.5,
#             )
#         ),
#         opacity=0.6
#     )
#     tracePreapproveDenied = graph_objs.Bar(
#       x= dfgraph[dfgraph['ActionType']==7].range, 
#       y= dfgraph[dfgraph['ActionType']==7].value,
#       name= "Preapproval request denied by financial institution", 
#           #orientation = 'h',
#     marker=dict(
#             color='rgb(121, 0, 0)',
#             line=dict(
#                 color='rgb(121, 0, 0)',
#                 width=1.5,
#             )
#         ),
#         opacity=0.6
#     )
#     tracePreapproveNotAccepted = graph_objs.Bar(
#       x= dfgraph[dfgraph['ActionType']==8].range, 
#       y= dfgraph[dfgraph['ActionType']==8].value,
#       name= "Preapproval request approved but not accepted", 
#           #orientation = 'h',
#     marker=dict(
#             color='rgb(135,206,250)',
#             line=dict(
#                 color='rgb(135,206,250)',
#                 width=1.5,
#             )
#         ),
#         opacity=0.6
#     )
#     data = Data([traceApprove,traceNotAccepted,traceDenied,tracePurchased, tracePreapproveDenied,tracePreapproveNotAccepted])


    layout = graph_objs.Layout(
        title=title,
        width=1050,
        height=950,
        yaxis=dict(
            tickfont=dict(
                size=10,
                color='rgb(107, 107, 107)'
            )
        ),
        xaxis=dict(
            title='Amount in (000)',
            titlefont=dict(
                size=16,
                color='rgb(107, 107, 107)'
            ),
            tickfont=dict(
                size=14,
                color='rgb(107, 107, 107)'
            )

        ),
    #    legend=dict(
    #        x=0,
    #        y=1.0,
    #        bgcolor='rgba(255, 255, 255, 0)',
    #        bordercolor='rgba(255, 255, 255, 0)'
    #    ),
        barmode='group',
        bargap=1.15,
        bargroupgap=1.55
    )
    #print((traceApprove.y.sum()))
    return data,layout,dfgraph 

#fig = Figure(data=data, layout=layout)
#return fig
#py.iplot(fig, filename='grouped-bar-County-salary')

In [161]:
# graph for 0-1101
# graph for 1101-5101
# graph for 5101-99999

def drawIncomegraphonCluster(df,title,start, end):
    #start=0 
    #end=1101    
    dfgraph=df[(np.int64(df.start)>=np.int64(start))& (np.int64(df.end)<=np.int64(end))].sort_values(by=['start'])
    #dfgraph=df_income_range[(np.int64(df_income_range.start)>np.int64(start)]
    #dfgraph=df_income_range[(np.int64(df_income_range.start)>np.int64(start))]                         
    #dfgraph=df_income_range[(np.int64(df_income_range.end)<=np.int64(end))].sort_values(by=['start'])
    
    traceCluster1 =graph_objs.Bar( 
      x= dfgraph[dfgraph['cluster_label']==1].range, 
      y= dfgraph[dfgraph['cluster_label']==1].value, 
      name= "Cluster 1",
          #orientation = 'h',
    marker=dict(
            color=colorCluster1,
            line=dict(
                color=colorCluster1,
                width=1.5,
            )
        ),
        opacity=0.6,

    )
    
    traceCluster2 = graph_objs.Bar(
      x= dfgraph[dfgraph['cluster_label']==2].range, 
      y= dfgraph[dfgraph['cluster_label']==2].value,
      name= "Cluster 2", 
          #orientation = 'h',
    marker=dict(
            color=colorCluster2,
            line=dict(
                color=colorCluster2,
                width=1.5,
            )
        ),
        opacity=0.6
    )
    traceCluster3 = graph_objs.Bar(
      x= dfgraph[dfgraph['cluster_label']==3].range, 
      y= dfgraph[dfgraph['cluster_label']==3].value,
      name= "Cluster 3", 
          #orientation = 'h',
    marker=dict(
            color=colorCluster3,
            line=dict(
                color=colorCluster3,
                width=1.5,
            )
        ),
        opacity=0.6
    )
   
    data = Data([traceCluster1,traceCluster2,traceCluster3])


    layout = graph_objs.Layout(
        title=title,
        width=1050,
        height=950,
        yaxis=dict(
            tickfont=dict(
                size=10,
                color='rgb(107, 107, 107)'
            )
        ),
        xaxis=dict(
            title='Amount in (000)',
            titlefont=dict(
                size=16,
                color='rgb(107, 107, 107)'
            ),
            tickfont=dict(
                size=14,
                color='rgb(107, 107, 107)'
            )

        ),
    #    legend=dict(
    #        x=0,
    #        y=1.0,
    #        bgcolor='rgba(255, 255, 255, 0)',
    #        bordercolor='rgba(255, 255, 255, 0)'
    #    ),
        barmode='group',
        bargap=1.15,
        bargroupgap=1.55
    )
    #print((traceApprove.y.sum()))
    return data,layout,dfgraph 

#fig = Figure(data=data, layout=layout)
#return fig
#py.iplot(fig, filename='grouped-bar-County-salary')

In [162]:
def createHalfGuagechart(base_chart, meter_chart,layout,filenameval):
    fig = {"data": [base_chart, meter_chart],
       "layout": layout }
    #py.iplot(fig, filename='CA-Cluster-half-chart-Male-County')
    #offline.plot(fig, filename=filenameval)
    createofflineChart(fig,filenameval)

In [163]:
def createPiChartForThreeValues(valuesfirst,valuessecond,valuethird,labels,title,filename,namefirst,namesecond,namethird):
    
#     domain1={'x':[0, 0.33], 'y':[0.66,1.0]}# for the cell (1,1)
#     domain2={'x':[0.66, 1], 'y':[0.66,1]}#cell (1,2)
#     domain3={'x':[0, 0.33], 'y':[0.33, 0.66]}#cell (2,1)
#     domain4={'x':[0.66, 1], 'y':[0.33, 0.66]}#cell (2,2)
#     domain5={'x':[0, 0.33], 'y':[0,0.33]}#cell (3,1)
#     domain6={'x': [0.66, 1], 'y':[0,0.33]}#cell (3,2)
    
    domain1={'x':[0, 0.1], 'y':[0,0.33]}# for the cell (1,1)
    domain2={'x':[0, 0.1], 'y':[0.33, 0.66]}#cell (1,2)
    domain3={'x':[0, 0.1], 'y':[0.66, 1]}#cell (1,3)
    
    fig = {
      "data": [
        {
          "values": valuesfirst,
          "labels":labels, 
          "text":[namefirst],
          "textposition":"inside",
          "domain": domain1,
          "name": namefirst,#"GHG Emissions",
          "hoverinfo":"label+percent+name",
          "hole": .4,
          "type": "pie"
        },
        {
          "values": valuessecond,
          "labels": labels,
          "text":[namesecond],
          "textposition":"inside",
          "domain": domain2,
          "name": namesecond,
          "hoverinfo":"label+percent+name",
          "hole": .4,
          "type": "pie"
        },
          {
          "values": valuethird,
          "labels": labels,
          "text":[namethird],
          "textposition":"inside",
          'domain': domain3,
          "name": namethird,
          "hoverinfo":"label+percent+name",
          "hole": .4,
          "type": "pie"
        }
      ],
      "layout": {
            "title":title,
            "annotations": [
                {
                    "font": {
                        "size": 20
                    },
                    "showarrow": False,
                    "text": namefirst,
                    "x": 0.15,
                    "y": 0.5
                },
                {
                    "font": {
                        "size": 20
                    },
                    "showarrow": False,
                    "text": namesecond,
                    "x": 0.65,
                    "y": 0.5
                },
                {
                    "font": {
                        "size": 20
                    },
                    "showarrow": False,
                    "text": namethird,
                    "x": 0.9,
                    "y": 0.5
                }
            ]
        }
    }
    return fig
    #return py.iplot(fig, filename='donut')

In [164]:
def createPiChartFortwoValues(valuesfirst,valuessecond,labels,title,filename,namefirst,namesecond):
    fig = {
      "data": [
        {
          "values": valuesfirst,#[16, 15, 12, 6, 5, 4, 42],
          "labels":labels, #[
#             "US",
#             "China",
#             "European Union",
#             "Russian Federation",
#             "Brazil",
#             "India",
#             "Rest of World"
#           ],
          "text":[namefirst],
          "textposition":"inside",
          "domain": {"x": [0, .48]},
          "name": namefirst,#"GHG Emissions",
          "hoverinfo":"label+percent+name",
          "hole": .4,
          "type": "pie"
        },
        {
          "values": valuessecond,
          "labels": labels,
          "text":[namesecond],
          "textposition":"inside",
          "domain": {"x": [.52, 1]},
          "name": namesecond,
          "hoverinfo":"label+percent+name",
          "hole": .4,
          "type": "pie"
        }
      ],
      "layout": {
            "title":title,
            "annotations": [
                {
                    "font": {
                        "size": 20
                    },
                    "showarrow": False,
                    "text": namefirst,
                    "x": 0.20,
                    "y": 0.5
                },
                {
                    "font": {
                        "size": 20
                    },
                    "showarrow": False,
                    "text": namesecond,
                    "x": 0.8,
                    "y": 0.5
                }
            ]
        }
    }
    return fig
    #py.iplot(fig, filename='donut')

In [165]:
def getDataFrameGroupByColForCounty(dfselect,colname,divisor=1000.0):
    dfgroup=df_selectdata.groupby([colname,'CountyName'], as_index=False)
    dfgraph=dfgroup.count().loc[:,[colname,'CountyName','Year']]
    dfgraph=dfgraph[dfgraph['Year'].isna()==False]
    dfgraph['Count']=((dfgraph['Year']/divisor)).apply(math.ceil)  
    return dfgraph

In [166]:
def getDataFrameGroupByColForCluster(dfselect,colname,divisor=1000.0):
    dfgroup=df_selectdata.groupby([colname,'cluster_label'], as_index=False)
    dfgraph=dfgroup.count().loc[:,[colname,'cluster_label','Year']]
    dfgraph=dfgraph[dfgraph['Year'].isna()==False]
    dfgraph['Count']=((dfgraph['Year']/divisor)).apply(math.ceil)  
    return dfgraph

In [167]:
def getbarForColumnForCountyGraph(name,yaxis,color):
    trace =graph_objs.Bar( 
      y= yaxis,#dfgraph[dfgraph['ApplicantSex']==2]['Count'], 
      x= Countylist, 
      name= name,
    marker=dict(
            color=color,
            line=dict(
                color=color,
                width=1.5,
            )
        ),
        opacity=0.6
    )
    return trace
    

In [168]:
def getbarForColumnForClusterGraph(name,yaxis,color):
    trace =graph_objs.Bar( 
      y= yaxis,#dfgraph[dfgraph['ApplicantSex']==2]['Count'], 
      x= Clusterlist, 
      name= name,
    marker=dict(
            color=color,
            line=dict(
                color=color,
                width=1.5,
            )
        ),
        opacity=0.6
    )
    return trace
    

In [169]:
def getcolorForCluster(clusterno):
    if(clusterno==1):
        return colorCluster1
    elif(clusterno==2):
        return colorCluster2
    elif(clusterno==3):
        return colorCluster3

### Bar graph for CA Cluster for each field

In [170]:
def getbargraphforCounty(data,title,filename,barmode='group',width=950):
    
    #data = Data([traceFemale, traceMale,traceUnknown])

    layout = graph_objs.Layout(
        title=title,
        width=width,
        xaxis=dict(
            tickfont=dict(
                size=10,
                color='rgb(107, 107, 107)'
            )
        ),
        yaxis=dict(
            title='Counts in(1000)',
            titlefont=dict(
                size=16,
                color='rgb(107, 107, 107)'
            ),
            tickfont=dict(
                size=14,
                color='rgb(107, 107, 107)'
            )
        ),
    #    legend=dict(
    #        x=0,
    #        y=1.0,
    #        bgcolor='rgba(255, 255, 255, 0)',
    #        bordercolor='rgba(255, 255, 255, 0)'
    #    ),
        barmode=barmode,
        bargap=0.15,
        bargroupgap=0.2
    )


    fig = Figure(data=data, layout=layout)
    #py.iplot(fig, filename='grouped-bar-gender')
    createofflineChart(fig,filename)    

#### Gender bar graph

In [39]:
colname='ApplicantSex'
#dfgraph=getDataFrameGroupByColForCounty(df_selectdata,colname)
dfgraph=getDataFrameGroupByColForCluster(df_selectdata,colname)

In [40]:
title='Loan counts for gender for three K-Mean cluster CA'
#labels=['American Indian or Alaska Native','Asian','Black or African American','Native Hawaiian or Other Pacific Islander','White','Information not provided ','Not applicable']
labels=['Men','Women','Unknown']
filename='CA-bar-chart-gender-3-Clusters'

# traceMale=getbarForColumnForCountyGraph(labels[0],dfgraph[dfgraph[colname]==1]['Count'],colorBlue2)
# traceFemale=getbarForColumnForCountyGraph(labels[1],dfgraph[dfgraph[colname]==2]['Count'],colorPink)
# traceUnknown=getbarForColumnForCountyGraph(labels[2],dfgraph[(dfgraph[colname]==3)|(dfgraph[colname]==4)]['Count'],colorGray)

traceMale=getbarForColumnForClusterGraph(labels[0],dfgraph[dfgraph[colname]==1]['Count'],colorBlue2)
traceFemale=getbarForColumnForClusterGraph(labels[1],dfgraph[dfgraph[colname]==2]['Count'],colorPink)
traceUnknown=getbarForColumnForClusterGraph(labels[2],dfgraph[(dfgraph[colname]==3)|(dfgraph[colname]==4)]['Count'],colorGray)


data = Data([traceFemale, traceMale,traceUnknown])
#getbargraphforCounty(data,'Loan counts for gender for three K-Mean cluster CA','Gender-LoanCount-Bar')
getbargraphforCounty(data,title,filename)

#### Race bar graph

In [41]:
colname='ApplicantRace'
#dfgraph=getDataFrameGroupByColForCounty(df_selectdata,colname)
dfgraph=getDataFrameGroupByColForCluster(df_selectdata,colname)

In [42]:
title='Loan counts for Race for three K-Mean cluster CA'
labels=['American Indian or Alaska Native','Asian','Black or African American','Native Hawaiian or Other Pacific Islander',
        'White','Information not provided ','Not applicable']
filename='CA-bar-chart-ApplicantRace-3-Clusters'

# traceAI=getbarForColumnForCountyGraph(labels[0],dfgraph[dfgraph[colname]==1]['Count'],colorYellow4)
# traceAs=getbarForColumnForCountyGraph(labels[1],dfgraph[dfgraph[colname]==2]['Count'],colorBlue2)
# traceBL=getbarForColumnForCountyGraph(labels[2],dfgraph[dfgraph[colname]==3]['Count'],colorGreen3)
# traceNH=getbarForColumnForCountyGraph(labels[3],dfgraph[dfgraph[colname]==4]['Count'],colorRed5)
# traceW=getbarForColumnForCountyGraph(labels[4],dfgraph[dfgraph[colname]==5]['Count'],colorOrange1)

# traceUnkown=getbarForColumnForCountyGraph(labels[5],dfgraph[(dfgraph[colname]==6)|(dfgraph[colname]==7)]['Count'],colorGray)

traceAI=getbarForColumnForClusterGraph(labels[0],dfgraph[dfgraph[colname]==1]['Count'],colorYellow4)
traceAs=getbarForColumnForClusterGraph(labels[1],dfgraph[dfgraph[colname]==2]['Count'],colorBlue2)
traceBL=getbarForColumnForClusterGraph(labels[2],dfgraph[dfgraph[colname]==3]['Count'],colorGreen3)
traceNH=getbarForColumnForClusterGraph(labels[3],dfgraph[dfgraph[colname]==4]['Count'],colorRed5)
traceW=getbarForColumnForClusterGraph(labels[4],dfgraph[dfgraph[colname]==5]['Count'],colorOrange1)

traceUnkown=getbarForColumnForClusterGraph(labels[5],dfgraph[(dfgraph[colname]==6)|(dfgraph[colname]==7)]['Count'],colorGray)


data = Data([traceAI, traceAs,traceBL,traceNH,traceW,traceUnkown])
getbargraphforCounty(data,title,filename,'stack',1250)
#getbargraphforCounty(data,title,filename,width=1250)

#### Ethinicity bar graph

In [43]:
colname='ApplicantEthnicity'
#dfgraph=getDataFrameGroupByColForCounty(df_selectdata,colname)
dfgraph=getDataFrameGroupByColForCluster(df_selectdata,colname)

In [44]:
title='Loan counts for Ethnicity for three K-Mean cluster CA'
labels=['Hispanic or Latino','Not Hispanic or Latino','Information not provided ','Not applicable']
filename='CA-bar-chart-Ethnicity-3-Clusters'

# traceHisp=getbarForColumnForCountyGraph(labels[0],dfgraph[dfgraph[colname]==1]['Count'],colorBlue2)
# traceNoHisp=getbarForColumnForCountyGraph(labels[1],dfgraph[dfgraph[colname]==2]['Count'],colorOrange1)
# traceUnkown=getbarForColumnForCountyGraph(labels[3],dfgraph[(dfgraph[colname]==3)]['Count'],colorGray)

traceHisp=getbarForColumnForClusterGraph(labels[0],dfgraph[dfgraph[colname]==1]['Count'],colorBlue2)
traceNoHisp=getbarForColumnForClusterGraph(labels[1],dfgraph[dfgraph[colname]==2]['Count'],colorOrange1)
traceUnkown=getbarForColumnForClusterGraph(labels[3],dfgraph[(dfgraph[colname]==3)]['Count'],colorGray)


data = Data([traceHisp, traceNoHisp,traceUnkown])
getbargraphforCounty(data,title,filename)
#getbargraphforCounty(data,title,filename,width=1250)

#### Action bar graph

In [45]:
colname='ActionType'
#dfgraph=getDataFrameGroupByColForCounty(df_selectdata,colname)
dfgraph=getDataFrameGroupByColForCluster(df_selectdata,colname)#

In [46]:
title='Loan counts for Action for three K-Mean cluster CA'
labels=['Loan originated' ,'Application approved but not accepted' , 'Application denied by financial institution', 
        'Application withdrawn by applicant', 'File closed for incompleteness',' Loan purchased by your institution',
        'Preapproval request denied by financial institution','Preapproval request approved but not accepted']
filename='CA-bar-chart-Action-3-Clusters'

# traceLO=getbarForColumnForCountyGraph(labels[0],dfgraph[dfgraph[colname]==1]['Count'],colorGreen3)
# traceApp=getbarForColumnForCountyGraph(labels[1],dfgraph[dfgraph[colname]==2]['Count'],colorBlue2)
# traceDen=getbarForColumnForCountyGraph(labels[2],dfgraph[dfgraph[colname]==3]['Count'],colorRed5)
# tracepur=getbarForColumnForCountyGraph(labels[5],dfgraph[dfgraph[colname]==6]['Count'],colorYellow4)
# tracePreDeny=getbarForColumnForCountyGraph(labels[6],dfgraph[dfgraph[colname]==7]['Count'],colorOrange1)
# tracePreNtAcc=getbarForColumnForCountyGraph(labels[7],dfgraph[dfgraph[colname]==8]['Count'],colorGray)

#traceUnkown=getbarForColumnForCountyGraph(labels[3],dfgraph[(dfgraph[colname]==4)|(dfgraph[colname]==5)]['Count'],'rgb(190,190,190)')

traceLO=getbarForColumnForClusterGraph(labels[0],dfgraph[dfgraph[colname]==1]['Count'],colorGreen3)
traceApp=getbarForColumnForClusterGraph(labels[1],dfgraph[dfgraph[colname]==2]['Count'],colorBlue2)
traceDen=getbarForColumnForClusterGraph(labels[2],dfgraph[dfgraph[colname]==3]['Count'],colorRed5)
tracepur=getbarForColumnForClusterGraph(labels[5],dfgraph[dfgraph[colname]==6]['Count'],colorYellow4)
tracePreDeny=getbarForColumnForClusterGraph(labels[6],dfgraph[dfgraph[colname]==7]['Count'],colorOrange1)
tracePreNtAcc=getbarForColumnForClusterGraph(labels[7],dfgraph[dfgraph[colname]==8]['Count'],colorGray)

data = Data([traceLO, traceApp,traceDen,tracepur,tracePreDeny,tracePreNtAcc])
getbargraphforCounty(data,title,filename,barmode='stack')
#getbargraphforCounty(data,title,filename,width=1250)

#### Loan Type bar graph # did not have data in select data

#### Loan Purpose bar graph

In [47]:
colname='LoanPurpose'
#dfgraph=getDataFrameGroupByColForCounty(df_selectdata,colname)
dfgraph=getDataFrameGroupByColForCluster(df_selectdata,colname)
df2 = pd.DataFrame([[1.0,0,0,0],[2.0,1,0,0],[3.0,1,0,0]],columns=['LoanPurpose','cluster_label','Year','Count'])
dfgraph=dfgraph.append(df2)#, columns=list('LoanPurposecluster_labelYearCount'))

In [48]:
title='Loan counts for Loan Purpose for three K-Mean cluster CA'
labels=['Home purchase',  'Home improvement', 'Refinancing']
filename='CA-bar-chart-LoanPurpose-3-Clusters'

# traceFam=getbarForColumnForCountyGraph(labels[0],dfgraph[dfgraph[colname]==1]['Count'],colorGreen3)
# traceMan=getbarForColumnForCountyGraph(labels[1],dfgraph[dfgraph[colname]==2]['Count'],colorBlue2)
# traceMulti=getbarForColumnForCountyGraph(labels[2],dfgraph[dfgraph[colname]==3]['Count'],colorRed5)
# #traceFSA=getbarForColumnForCountyGraph(labels[2],dfgraph[dfgraph[colname]==4]['Count'],colorYellow4)


traceFam=getbarForColumnForClusterGraph(labels[0],dfgraph[dfgraph[colname]==1]['Count'],colorGreen3)
traceMan=getbarForColumnForClusterGraph(labels[1],dfgraph[dfgraph[colname]==2]['Count'],colorBlue2)
traceMulti=getbarForColumnForClusterGraph(labels[2],dfgraph[dfgraph[colname]==3]['Count'],colorRed5)
#traceFSA=getbarForColumnForCountyGraph(labels[2],dfgraph[dfgraph[colname]==4]['Count'],colorYellow4)


data = Data([traceFam, traceMan,traceMulti])
getbargraphforCounty(data,title,filename,barmode='group')
#getbargraphforCounty(data,title,filename,width=1250)

#### Property Type bar graph

In [49]:
colname='PropertyType'
#dfgraph=getDataFrameGroupByColForCounty(df_selectdata,colname)
dfgraph=getDataFrameGroupByColForCluster(df_selectdata,colname)

In [50]:
title='Loan counts for Property Type for three K-Mean cluster CA'
labels=['One to four-family',  'Manufactured housing', 'Multifamily']
filename='CA-bar-chart-PropertyType-3-Clusters'

# traceFam=getbarForColumnForCountyGraph(labels[0],dfgraph[dfgraph[colname]==1]['Count'],colorGreen3)
# traceMan=getbarForColumnForCountyGraph(labels[1],dfgraph[dfgraph[colname]==2]['Count'],colorBlue2)
# traceMulti=getbarForColumnForCountyGraph(labels[2],dfgraph[dfgraph[colname]==3]['Count'],colorRed5)
# #traceFSA=getbarForColumnForCountyGraph(labels[2],dfgraph[dfgraph[colname]==4]['Count'],colorYellow4)

traceFam=getbarForColumnForClusterGraph(labels[0],dfgraph[dfgraph[colname]==1]['Count'],colorGreen3)
traceMan=getbarForColumnForClusterGraph(labels[1],dfgraph[dfgraph[colname]==2]['Count'],colorBlue2)
traceMulti=getbarForColumnForClusterGraph(labels[2],dfgraph[dfgraph[colname]==3]['Count'],colorRed5)
#traceFSA=getbarForColumnForClusterGraph(labels[2],dfgraph[dfgraph[colname]==4]['Count'],colorYellow4)

data = Data([traceFam, traceMan,traceMulti])
getbargraphforCounty(data,title,filename,barmode='group')
#getbargraphforCounty(data,title,filename,width=1250)

#### Owner Occupancy bar graph

In [51]:
colname='Occupancy'
dfgraph=getDataFrameGroupByColForCluster(df_selectdata,colname)

In [52]:
title='Loan counts for Owner Occupancy for three K-Mean cluster CA'
labels=['Owner-occupied as a principal dwelling',  'Not owner-occupied', 'Not applicable']
filename='CA-bar-chart-OwnerOccupancy-3-Clusters'

traceFam=getbarForColumnForCountyGraph(labels[0],dfgraph[dfgraph[colname]==1]['Count'],colorGreen3)
traceMan=getbarForColumnForCountyGraph(labels[1],dfgraph[dfgraph[colname]==2]['Count'],colorBlue2)
traceMulti=getbarForColumnForCountyGraph(labels[2],dfgraph[dfgraph[colname]==3]['Count'],colorGray)
#traceFSA=getbarForColumnForCountyGraph(labels[2],dfgraph[dfgraph[colname]==4]['Count'],colorYellow4)

data = Data([traceFam, traceMan,traceMulti])
getbargraphforCounty(data,title,filename,barmode='group')
#getbargraphforCounty(data,title,filename,width=1250)

### Action and County bar

In [53]:
dfgroup=df_selectdata.groupby(['cluster_label','CountyName'], as_index=False)
dfgraph=dfgroup.count().loc[:,['cluster_label','CountyName','Year']]
#dfgraph=dfgraph[dfgraph['Year'].isna()==False]
dfgraph['Count']=((dfgraph['Year']/1000.0)).apply(math.ceil)

In [54]:
traceCluster1 =graph_objs.Bar( 
  x= dfgraph[dfgraph['cluster_label']==1]['Count'], 
  y= Countylist, 
  name= "Cluster 1",
      orientation = 'h',
marker=dict(
        color=colorCluster1,
        line=dict(
            color=colorCluster1,
            width=1.5,
        )
    ),
    opacity=0.6,
    
)
traceCluster2 = graph_objs.Bar(
  x= dfgraph[dfgraph['cluster_label']==2]['Count'], 
  y= Countylist,
  name= "Cluster 2", 
      orientation = 'h',
marker=dict(
        color=colorCluster2,
        line=dict(
            color=colorCluster2,
            width=1.5,
        )
    ),
    opacity=0.6
)
traceCluster3 = graph_objs.Bar(
  x= dfgraph[(dfgraph['cluster_label']==3)]['Count'], 
  y= Countylist,
  name= "Cluster 3", 
      orientation = 'h',
marker=dict(
        color=colorCluster3,
        line=dict(
            color=colorCluster3,
            width=1.5,
        )
    ),
    opacity=0.6
)

data = Data([traceCluster1,traceCluster2,traceCluster3])

layout = graph_objs.Layout(
    title='Loan counts for cluster for three K-Mean cluster CA',
    width=950,
    height=950,
    yaxis=dict(
        tickfont=dict(
            size=10,
            color='rgb(107, 107, 107)'
        )
        ,autorange= True, 
    ),
    xaxis=dict(
        title='Counts (000)',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
        ,autorange= True, 
    ),
#    legend=dict(
#        x=0,
#        y=1.0,
#        bgcolor='rgba(255, 255, 255, 0)',
#        bordercolor='rgba(255, 255, 255, 0)'
#    ),
    barmode='stack',
    bargap=0.15,
    bargroupgap=0.2
)


fig = Figure(data=data, layout=layout)
#py.iplot(fig, filename='grouped-bar-County')
createofflineChart(fig,'grouped-bar-CA-County-KMean-cluster')

### Histogram graph

#### Income & action bar

In [55]:
def getClusterAmountRangeDataFrame(clusterno,amountcol):
    # The logic to conver the income by range with action type
    count=-1
    rangestart=0
    rangeend=50
    gap=50;
    i=0
    #clusterno=0
    df_income_range=  pd.DataFrame(columns=['range', 'start', 'end','value'])
#     action_list=df_selectdata.ActionType.unique()#[1,2,3,4,5,6,7,8,9]
    df_cluster=df_selectdata[df_selectdata.cluster_label==clusterno]
    while(rangeend<10000):
       # df=  pd.DataFrame()
        #df['range']=str.format("{}-{}",rangestart,rangeend)
       # df['start']=rangestart
        #df['end']=rangeend    
        dfcount=getCountForAmountRange(df_cluster,amountcol,rangestart,rangeend)
        for index, row in dfcount.iterrows():
            count=row[0]
            df_income_range.loc[i] =((str.format("{}-{}",rangestart,rangeend)),rangestart,rangeend,count)
            i=i+1
#             for j in pd.Series(action_list):
#                 actiontype=j
#                 if j==index:
#                     count=row[0]
#                 else:
#                     count=0
#                 df_income_range.loc[i] =((str.format("{}-{}",rangestart,rangeend)),rangestart,rangeend,actiontype,count)
#                 i=i+1
        if(rangeend>=500):
            gap=5000
        rangestart=rangeend
        rangeend=rangeend+gap
            #df_income_range=df_income_range.append(df, ignore_index=True)
            #i=i+1


    #df_income_range=df_income_range.append(df, ignore_index=True)
    #df_income_range.loc[(len(df_income_range))] =(str.format("{}-{}",rangeend,999999),rangeend,999999,y[(y.ApplicantIncome>rangeend)].Year.count())
    i=len(df_income_range)
    #print('Now '+str(i))
    dfcount=getCountForAmountRange(df_cluster,amountcol,rangeend,999999)
    for index, row in dfcount.iterrows():
        df_income_range.loc[i] =((str.format("{}-{}",rangestart,999999)),rangestart,999999,count)
        i=i+1

#         for j in pd.Series(action_list):
#                 actiontype=j
#                 if j==index:
#                     count=row[0]
#                 else:
#                     count=0
#                 df_income_range.loc[i] =((str.format("{}-{}",rangestart,999999)),rangestart,999999,actiontype,count)
#                 i=i+1

    #df_income_m_30=y[(y.ApplicantIncome>rangeend)].Year.count()
    #print(df_income_m_30)   
    df_income_range=df_income_range.groupby(['start','end'],as_index=False).sum()    
    df_income_range['range']=df_income_range['start'].astype(str) + '-'+df_income_range['end'].astype(str) 
    return df_income_range

In [56]:
def getClusterAmountRangeDataFrameWithCluster(amountcol):
    # The logic to conver the income by range with action type
    count=-1
    rangestart=0
    rangeend=50
    gap=50;
    i=0
    #clusterno=0
    df_income_range=  pd.DataFrame(columns=['range', 'start', 'end','cluster_label','value'])
#     action_list=df_selectdata.ActionType.unique()#[1,2,3,4,5,6,7,8,9]
    
    while(rangeend<10000):
       # df=  pd.DataFrame()
        #df['range']=str.format("{}-{}",rangestart,rangeend)
       # df['start']=rangestart
        #df['end']=rangeend    
        #dfcount=getCountForAmountRange(df_cluster,'ApplicantIncome',rangestart,rangeend)
        
        dfcount=getCountForAmountRange(df_selectdata,amountcol,rangestart,rangeend,'cluster_label')
        for index, row in dfcount.iterrows():
            for j in pd.Series(Clusterlist):
                actiontype=j
                if j==index:
                    count=row[0]
                else:
                    count=0
                df_income_range.loc[i] =((str.format("{}-{}",rangestart,rangeend)),rangestart,rangeend,actiontype,count)
                i=i+1
        if(rangeend>=500):
            gap=5000
        rangestart=rangeend
        rangeend=rangeend+gap
            #df_income_range=df_income_range.append(df, ignore_index=True)
            #i=i+1


    #df_income_range=df_income_range.append(df, ignore_index=True)
    #df_income_range.loc[(len(df_income_range))] =(str.format("{}-{}",rangeend,999999),rangeend,999999,y[(y.ApplicantIncome>rangeend)].Year.count())
    i=len(df_income_range)
    #print('Now '+str(i))
    dfcount=getCountForAmountRange(df_selectdata,amountcol,rangeend,999999,'cluster_label')
    for index, row in dfcount.iterrows():
        for j in pd.Series(Clusterlist):
                actiontype=j
                if j==index:
                    count=row[0]
                else:
                    count=0
                df_income_range.loc[i] =((str.format("{}-{}",rangestart,999999)),rangestart,999999,actiontype,count)
                i=i+1

    #df_income_m_30=y[(y.ApplicantIncome>rangeend)].Year.count()
    #print(df_income_m_30)   
    df_income_range=df_income_range.groupby(['start','end','cluster_label'],as_index=False).sum()    
    df_income_range['range']=df_income_range['start'].astype(str) + '-'+df_income_range['end'].astype(str) 
    return df_income_range

In [57]:
for clusterno in Clusterlist:
    df_cluster_range=getClusterAmountRangeDataFrame(clusterno,'ApplicantIncome')
    data,layout,dfgraph=drawIncomegraphonRange(df_cluster_range,'Cluster '+str(clusterno)+'  grouped by salary range'
                                               ,0, 999999,getcolorForCluster(clusterno))
    fig = Figure(data=data, layout=layout)
    #py.iplot(fig, filename='grouped-bar-County-salary')
    createofflineChart(fig,'CA-Cluster'+str(clusterno)+'-grouped-bar-Salary')

In [58]:
df_cluster_range=getClusterAmountRangeDataFrameWithCluster('ApplicantIncome')
data,layout,dfgraph=drawIncomegraphonCluster(df_cluster_range,'CA clusters grouped by Salary range',0, 999999)
fig = Figure(data=data, layout=layout)
#py.iplot(fig, filename='grouped-bar-state-salary')
createofflineChart(fig,'CA-All-cluster3-grouped-bar-salary')

#### loan amount County graph

In [59]:
for clusterno in Clusterlist:
    df_cluster_range=getClusterAmountRangeDataFrame(clusterno,'LoanAmount')
    data,layout,dfgraph=drawIncomegraphonRange(df_cluster_range,'Cluster '+str(clusterno)+'  grouped by loan amount range'
                                               ,0, 999999,getcolorForCluster(clusterno))
    fig = Figure(data=data, layout=layout)
    #py.iplot(fig, filename='grouped-bar-County-salary')
    createofflineChart(fig,'CA-Cluster'+str(clusterno)+'-grouped-bar-loanamt')

In [60]:
df_cluster_range=getClusterAmountRangeDataFrameWithCluster('LoanAmount')
data,layout,dfgraph=drawIncomegraphonCluster(df_cluster_range,'CA clusters grouped by loan amount range',0, 999999)
fig = Figure(data=data, layout=layout)
#py.iplot(fig, filename='grouped-bar-state-salary')
createofflineChart(fig,'CA-All-cluster3-grouped-bar-loanamt')

### Half pie graph for Cluster for each field

#### Race half pie graph

In [61]:
colname='ApplicantRace'
#dfselectRace=getDataFrameGrroupedbyCol(df_selectdata,colname)
dfselectRace=getDataFrameGrroupedbyClusterAndCol(df_selectdata,colname)

In [62]:
### working on making 1 file 4 graph
#race=1
#raceName='American Indian or Alaska Native'
title='CA 3 Clusters by Applicant Race'
#labellist=[raceName,"Denied","Accepted"]
labels=['American Indian or Alaska Native','Asian','Black or African American','Native Hawaiian or Other Pacific Islander','White','Information not provided/NA','Not applicable']
filename='CA-Cluster-half-chart-Race'
inputPercList1,inputPercListText1=getlistbycolumnForCluster(dfselectRace,colname,1)
inputPercList2,inputPercListText2=getlistbycolumnForCluster(dfselectRace,colname,2)
inputPercList3,inputPercListText3=getlistbycolumnForCluster(dfselectRace,colname,3)
inputPercList4,inputPercListText4=getlistbycolumnForCluster(dfselectRace,colname,4)
inputPercList5,inputPercListText5=getlistbycolumnForCluster(dfselectRace,colname,5)
inputPercList6,inputPercListText6=getlistbycolumnForCluster(dfselectRace,colname,[6,7])

createSixHalfDonut(title,inputPercList1,inputPercListText1,inputPercList2,inputPercListText2,
                        inputPercList3,inputPercListText3,inputPercList4,inputPercListText4,inputPercList5,inputPercListText5,
                   inputPercList6,inputPercListText6,labels,filename)

#### Gender half pie graph

In [63]:
colname='ApplicantSex'
# dfselect=getDataFrameGrroupedbyCol(df_selectdata,colname)
dfselect=getDataFrameGrroupedbyClusterAndCol(df_selectdata,colname)

In [64]:
title='CA 3 clusters by Applicant Gender'
labels=['Men','Women','Unknown']
filename='CA-Cluster-half-chart-gender'
inputPercList1,inputPercListText1=getlistbycolumnForCluster(dfselect,colname,1)
inputPercList2,inputPercListText2=getlistbycolumnForCluster(dfselect,colname,2)
inputPercList3,inputPercListText3=getlistbycolumnForCluster(dfselect,colname,[3,4])

createThreeHalfDonut(title,inputPercList1,inputPercListText1,inputPercList2,inputPercListText2,
                        inputPercList3,inputPercListText3,labels,filename)

#### Ethinicity half pie graph

In [65]:
colname='ApplicantEthnicity'
# dfselect=getDataFrameGrroupedbyCol(df_selectdata,colname)
dfselect=getDataFrameGrroupedbyClusterAndCol(df_selectdata,colname)

In [66]:
title='CA 3 clusters by Applicant Ethinicity'
labels=['Hispanic or Latino','Not Hispanic or Latino','Information not provided ','Not applicable']
filename='CA-Cluster-half-chart-Ethnicity'
inputPercList1,inputPercListText1=getlistbycolumnForCluster(dfselect,colname,1)
inputPercList2,inputPercListText2=getlistbycolumnForCluster(dfselect,colname,2)
inputPercList3,inputPercListText3=getlistbycolumnForCluster(dfselect,colname,[3,4])

createThreeHalfDonut(title,inputPercList1,inputPercListText1,inputPercList2,inputPercListText2,
                        inputPercList3,inputPercListText3,labels,filename)

#### Loan Type half pie graph # we dont have data

#### Property Type half pie graph

In [67]:
colname='PropertyType'
# dfselect=getDataFrameGrroupedbyCol(df_selectdata,colname)
dfselect=getDataFrameGrroupedbyClusterAndCol(df_selectdata,colname)

In [68]:
title='CA 3 clusters by Applicant Properrt Type'
labels=['One to four-family',  'Manufactured housing', 'Multifamily']
filename='CA-Cluster-half-chart-PropertyType'
inputPercList1,inputPercListText1=getlistbycolumnForCluster(dfselect,colname,1)
inputPercList2,inputPercListText2=getlistbycolumnForCluster(dfselect,colname,2)
inputPercList3,inputPercListText3=getlistbycolumnForCluster(dfselect,colname,3)


createThreeHalfDonut(title,inputPercList1,inputPercListText1,inputPercList2,inputPercListText2,
                        inputPercList3,inputPercListText3,labels,filename)

#### Owner Occupancy half pie graph

In [69]:
colname='Occupancy'
# dfselect=getDataFrameGrroupedbyCol(df_selectdata,colname)
dfselect=getDataFrameGrroupedbyClusterAndCol(df_selectdata,colname)

In [70]:
title='CA 3 clusters by Occupancy'
labels=['Owner-occupied as a principal dwelling',  'Not owner-occupied', 'Not applicable']
filename='CA-Cluster-half-chart-OwnerOccupancy'
inputPercList1,inputPercListText1=getlistbycolumnForCluster(dfselect,colname,1)
inputPercList2,inputPercListText2=getlistbycolumnForCluster(dfselect,colname,2)
inputPercList3,inputPercListText3=getlistbycolumnForCluster(dfselect,colname,3)


createThreeHalfDonut(title,inputPercList1,inputPercListText1,inputPercList2,inputPercListText2,
                        inputPercList3,inputPercListText3,labels,filename)

#### Loan Purpose half pie graph

In [71]:
colname='LoanPurpose'
# dfselect=getDataFrameGrroupedbyCol(df_selectdata,colname)
dfselect=getDataFrameGrroupedbyClusterAndCol(df_selectdata,colname)

In [72]:
title='CA 3 clusters by Loan Purpose'
labels=['Home purchase',  'Home improvement', 'Refinancing']
filename='CA-Cluster-half-chart-LoanPurpose'
inputPercList1,inputPercListText1=getlistbycolumnForCluster(dfselect,colname,1)
inputPercList2,inputPercListText2=getlistbycolumnForCluster(dfselect,colname,2)
inputPercList3,inputPercListText3=getlistbycolumnForCluster(dfselect,colname,3)


createThreeHalfDonut(title,inputPercList1,inputPercListText1,inputPercList2,inputPercListText2,
                        inputPercList3,inputPercListText3,labels,filename)

### Donut graph for each field

In [172]:
def createDonutChart(value,labels,title,name):
    fig = {
      "data": [
        {
          "values": value,
          "labels":labels, 
          #"text":text,
          "textposition":"inside",
          "domain": {"x": [0, .48]},
          "name": name,
          "hoverinfo":"label+name",#"label+percent+name",
          "hole": .4,
          "type": "pie",
          "textinfo":"value"
        
        }
        
      ],
      "layout": {
            "title":title,
            "annotations": [
                {
                    "font": {
                        "size": 20
                    },
                    "showarrow": False,
                    "text": name,
                    "x": 0.20,
                    "y": 0.5
                },

            ]
        }
    }
    return fig
    #py.iplot(fig, filename='donut')

In [174]:
#gender
colname='ApplicantSex'
# dfselectProp=getDataFrameGrroupedbyCol(df_selectdata,colname)
dfselectProp=getDataFrameGrroupedbyClusterAndCol(df_selectdata,colname)

In [175]:
# inputPercList1,inputPercListText1=getlistofCusterbycolumn(dfselectProp,1)
# inputPercList2,inputPercListText2=getlistofCusterbycolumn(dfselectProp,2)
# inputPercList3,inputPercListText2=getlistofCusterbycolumn(dfselectProp,3)
# namefirst='1'
# namesecond='2'
# namethird='3'
title='Distribution by Applicant gender for cluster '
labels=['Male','Female','Information not provided ','Not applicable']
filename='CA-donut-chart-gender-Cluster-'
# fig=createPiChartFortwoValues(inputPercList1,inputPercList2,labels,title,filename,namefirst,namesecond)

for clusr in Clusterlist:
    inputPercList,inputPercListText=getlistofCusterbycolumn(dfselectProp,clusr)
    fig=createDonutChart(inputPercList,labels,title+str(clusr),'Cluster-'+str(clusr))
    createofflineChart(fig,filename+str(clusr))

#### Race donut chart

In [176]:
# owner ApplicantRace
colname='ApplicantRace'
dfselectProp=getDataFrameGrroupedbyClusterAndCol(df_selectdata,colname)

In [177]:
# inputPercList1,inputPercListText1=getlistofCusterbycolumn(dfselectProp,0)
# inputPercList2,inputPercListText2=getlistofCusterbycolumn(dfselectProp,1)
# namefirst='Approved'
# namesecond='Denied'
title='Distribution by Applicant Race for cluster '
labels=['American Indian or Alaska Native','Asian','Black or African American','Native Hawaiian or Other Pacific Islander','White','Information not provided ','Not applicable']
filename='CA-donut-chart-ApplicantRace-Cluster-'
# fig=createPiChartFortwoValues(inputPercList1,inputPercList2,labels,title,filename,namefirst,namesecond)
# createofflineChart(fig,filename)

for clusr in Clusterlist:
    inputPercList,inputPercListText=getlistofCusterbycolumn(dfselectProp,clusr)
    fig=createDonutChart(inputPercList,labels,title+str(clusr),'Cluster-'+str(clusr))
    createofflineChart(fig,filename+str(clusr))

#### Ethinicity donut chart

In [178]:
# owner ethincity
colname='ApplicantEthnicity'
dfselectProp=getDataFrameGrroupedbyClusterAndCol(df_selectdata,colname)

In [179]:
# inputPercList1,inputPercListText1=getlistofCusterbycolumn(dfselectProp,0)
# inputPercList2,inputPercListText2=getlistofCusterbycolumn(dfselectProp,1)
# namefirst='Approved'
# namesecond='Denied'
title='Distribution by Applicant Ethnicity for cluster '
labels=['Hispanic or Latino','Not Hispanic or Latino','Information not provided ','Not applicable']
filename='CA-donut-chart-Ethin-Cluster-'
# fig=createPiChartFortwoValues(inputPercList1,inputPercList2,labels,title,filename,namefirst,namesecond)
# createofflineChart(fig,filename)


for clusr in Clusterlist:
    inputPercList,inputPercListText=getlistofCusterbycolumn(dfselectProp,clusr)
    fig=createDonutChart(inputPercList,labels,title+str(clusr),'Cluster-'+str(clusr))
    createofflineChart(fig,filename+str(clusr))

#### Property Type donut chart

In [180]:
# property type
colname='PropertyType'
dfselectProp=getDataFrameGrroupedbyClusterAndCol(df_selectdata,colname)

In [181]:
# # res=1
# # propname='One to four-family'
# inputPercList1,inputPercListText1=getlistofCusterbycolumn(dfselectProp,0)
# inputPercList2,inputPercListText2=getlistofCusterbycolumn(dfselectProp,1)
# namefirst='Approved'
# namesecond='Denied'
title='Distribution by Loan Property for cluster '
labels=['One to four-family (other than manufactured housing)','Manufactured housing','Multifamily']
filename='CA-donut-chart-propType-Cluster-'
# fig=createPiChartFortwoValues(inputPercList1,inputPercList2,labels,title,filename,namefirst,namesecond)
# createofflineChart(fig,filename)

for clusr in Clusterlist:
    inputPercList,inputPercListText=getlistofCusterbycolumn(dfselectProp,clusr)
    fig=createDonutChart(inputPercList,labels,title+str(clusr),'Cluster-'+str(clusr))
    createofflineChart(fig,filename+str(clusr))

#### Owner Occupancy donut chart

In [182]:
# owner occupancy
colname='Occupancy'
dfselectProp=getDataFrameGrroupedbyClusterAndCol(df_selectdata,colname)

In [183]:
# inputPercList1,inputPercListText1=getlistofCusterbycolumn(dfselectProp,0)
# inputPercList2,inputPercListText2=getlistofCusterbycolumn(dfselectProp,1)
# namefirst='Approved'
# namesecond='Denied'
title='Distribution by Owner Occupancy for cluster '
labels=['Owner-occupied as a principal dwelling','Not owner-occupied','Not applicable']
filename='CA-donut-chart-OwnerOccupancy-Cluster-'
# fig=createPiChartFortwoValues(inputPercList1,inputPercList2,labels,title,filename,namefirst,namesecond)
# createofflineChart(fig,filename)

for clusr in Clusterlist:
    inputPercList,inputPercListText=getlistofCusterbycolumn(dfselectProp,clusr)
    fig=createDonutChart(inputPercList,labels,title+str(clusr),'Cluster-'+str(clusr))
    createofflineChart(fig,filename+str(clusr))

#### Loan Purpose donut chart

In [184]:
# loan purpose
colname='LoanPurpose'
dfselectProp=getDataFrameGrroupedbyClusterAndCol(df_selectdata,colname)

In [185]:
# inputPercList1,inputPercListText1=getlistofCusterbycolumn(dfselectProp,0)
# inputPercList2,inputPercListText2=getlistofCusterbycolumn(dfselectProp,1)
# namefirst='Approved'
# namesecond='Denied'
title='Distribution by Loan Purpose for cluster '
labels=['Home purchase','Home improvement','Refinancing']
filename='CA-donut-chart-LoanPurp-Cluster-'
# fig=createPiChartFortwoValues(inputPercList1,inputPercList2,labels,title,filename,namefirst,namesecond)
# createofflineChart(fig,filename)

for clusr in Clusterlist:
    inputPercList,inputPercListText=getlistofCusterbycolumn(dfselectProp,clusr)
    fig=createDonutChart(inputPercList,labels,title+str(clusr),'Cluster-'+str(clusr))
    createofflineChart(fig,filename+str(clusr))

## box plot

In [87]:
def drawThreeWhiskerplot(inc1,inc2,inc3,axisrange,titletext,filename):
    
    
    # "all" | "outliers" | "suspectedoutliers" | False )
    boxtype=False
    cluster1 = graph_objs.Box(
        x = inc1,
        name = "Cluster 1",
       jitter = 0.3,
       pointpos = -1.8,
       boxpoints = boxtype,
        marker = dict(
            color = colorCluster1),
        line = dict(
            color =  colorCluster1)
    )

    cluster2 = graph_objs.Box(
        x = inc2,
        name = "Cluster 2",
       jitter = 0.3,
       pointpos = -1.8,
       boxpoints = boxtype,
        marker = dict(
            color =  colorCluster2),
        line = dict(
            color =  colorCluster2)
    )
    cluster3 = graph_objs.Box(
        #mode= "markers+text",
        x = inc3,
        name = "Cluster 3",
       jitter = 0.3,
       pointpos = -1.8,
       boxpoints = boxtype,
        marker = dict(
            color =  colorCluster3),
        line = dict(
            color =  colorCluster3)
    )


    # trace1 = graph_objs.Box(
    #     x = inc2,
    #     name = "Only Whiskers",
    #     boxpoints = False,
    #     marker = dict(
    #         color = 'rgb(9,56,125)'),
    #     line = dict(
    #         color = 'rgb(9,56,125)')
    # )

    # trace2 = graph_objs.Box(
    #     x = inc2,
    #     name = "Suspected Outliers",
    #     boxpoints = 'suspectedoutliers',
    #     marker = dict(
    #         color = 'rgb(8,81,156)',
    #         outliercolor = 'rgba(219, 64, 82, 0.6)',
    #         line = dict(
    #             outliercolor = 'rgba(219, 64, 82, 0.6)',
    #             outlierwidth = 2)),
    #     line = dict(
    #         color = 'rgb(8,81,156)')
    # )
    # trace3 = graph_objs.Box(
    #     x = inc2,
    #     name = "Whiskers and Outliers",
    #     boxpoints = 'outliers',
    #     marker = dict(
    #         color = 'rgb(107,174,214)'),
    #     line = dict(
    #         color = 'rgb(107,174,214)')
    # )
    # data = [trace0,trace1,trace2,trace3]
    data = [cluster1,cluster2,cluster3]
    layout = graph_objs.Layout(
        title =titletext,
         xaxis=dict(
    #         domain=[10,20]
             range=axisrange
        ),
       # showlegend=False

    #     margin = dict(
    #     r = 10,
    #     t = 25,
    #     b = 40,
    #     l = 110
    #   ),
#         annotations=[
#         dict(
#             x=2,
#             y=2,
#             xref='x',
#             yref='y',
#             text=2,
#             showarrow=True,
#             arrowhead=7,
#             ax=0,
#             ay=-40
#         )
#     ]
    )

    fig = graph_objs.Figure(data=data,layout=layout)

    createofflineChart(fig,filename)

In [88]:
colname='ApplicantIncome'
#dfgraph=getDataFrameGroupByColForCounty(df_selectdata,colname)
dfgraph=getDataFrameGroupByColForCluster(df_selectdata,colname)

In [89]:
inc1=df_selectdata[df_selectdata.cluster_label==1].ApplicantIncome
inc2=df_selectdata[df_selectdata.cluster_label==2].ApplicantIncome
inc3=df_selectdata[df_selectdata.cluster_label==3].ApplicantIncome
drawThreeWhiskerplot(inc1,inc2,inc3,[0, 500] ,"CA Cluster 3 Box Plot Applicant Income","CA-Cluster-3-Box-Plot-Applicant-Income")

In [90]:
colname='LoanAmount'
#dfgraph=getDataFrameGroupByColForCounty(df_selectdata,colname)
dfgraph=getDataFrameGroupByColForCluster(df_selectdata,colname)

In [91]:
lao1=df_selectdata[df_selectdata.cluster_label==1].LoanAmount
lao2=df_selectdata[df_selectdata.cluster_label==2].LoanAmount
lao3=df_selectdata[df_selectdata.cluster_label==3].LoanAmount
drawThreeWhiskerplot(lao1,lao2,lao3,[0, 1000] ,"CA Cluster 3 Box Plot Loan Amount","CA-Cluster-3-Box-Plot-LoanAmount")

## scatter plot

In [92]:
def getScatterpoints(x,y,coloval,nameval,symboltype=0,sizeval=12,opacityval=1,modeval="lines+markers"):
    return {
      "x": x, 
      "y": y, 
      "marker": {
        "color": coloval, 
        "size": sizeval,
#         "symbols": 'x'
          'symbol': symboltype#104
      }, 
      "mode": modeval, 
      "name": nameval, 
      "type": "scatter", 
      "uid": "cfa542",
        "opacity":opacityval
    }
#     return graph_objs.Scattergl(
#           x= inc1, 
#           y= lao1, 
#           marker= dict(
#             color= coloval, 
#             size= 12
#           ), 
#         symbols = 'x',#c('circle','x','o'),
#           mode= modeval, 
#           name= nameval, 
#           type= "scatter", 
#           #"uid": "cfa542"
#     )
   

In [93]:
def createScatterplotforOne(trace,titleval,filename):
    data = Data([trace])
    layout = {
      "autosize": True, 
      #"height": 650, 
      #"hidesources": True, 
      #"hovermode": "closest", 
      "margin": {"l": 100}, 
      "title": titleval, 
      #"width": 950, 
      "xaxis": {
        "autorange": True, 
        #"range": [10, 1000], 
        "title": "Salary Amount(in thousand )", 
        "type": "linear"
      }, 
      "yaxis": {
        "autorange": True, 
        #"range": [-1.51960784314, 21.5196078431], 
        "title": "Loan Amount(in thousand )", 
        "type": "linear"
      }
    }
    fig = Figure(data=data, layout=layout)
    createofflineChart(fig,filename)

In [94]:
filename='CA-Scatter-Loan-Salary-Cluster-'
for c in Clusterlist:
    df_graph=df_selectdata[df_selectdata.cluster_label==c]#.iloc[:100,:]
    
    tracecluster=getScatterpoints(df_graph.ApplicantIncome,df_graph.LoanAmount,getcolorForCluster(c),"Cluster "+str(c))
    createScatterplotforOne(tracecluster,"Income vs loan amount for CA cluster "+str(c) ,filename+str(c))
    

In [95]:
#getScatterpoints(x,y,coloval,nameval,symboltype=0,sizeval=12,opacityval=1,modeval="lines+markers")
# x1,y1,x2,y2,x3,y3=inc1.head(),lao1.head(),inc2.head(),lao2.head(),inc3.head(),lao3.head(),
x1,y1,x2,y2,x3,y3=inc1,lao1,inc2,lao2,inc3,lao3

tracecluster1=getScatterpoints(x1,y1,colorCluster1,"Cluster 1",0,8,1,"markers")
tracecluster2=getScatterpoints(x2,y2,colorCluster2,"Cluster 2",103,15,1,"markers")
tracecluster3=getScatterpoints(x3,y3,colorCluster3,"Cluster 3",121,19,1,"markers")
data = Data([tracecluster1, tracecluster2,tracecluster3])
layout = {
  "autosize": True, 
  #"height": 650, 
  #"hidesources": True, 
  #"hovermode": "closest", 
  "margin": {"l": 100}, 
  "title": "Income vs loan amount for CA cluster", 
  #"width": 950, 
  "xaxis": {
    "autorange": False, 
    "range": [0, 100], 
    "title": "Salary Amount(in thousand )", 
    "type": "linear"
  }, 
  "yaxis": {
    "autorange": False, 
    "range": [0, 200], 
    "title": "Loan Amount(in thousand )", 
    "type": "linear",
      
  },
    
}
fig = Figure(data=data, layout=layout)
createofflineChart(fig,"CA-Salary-LoanAmt-For-Cluster")